In [8]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import joblib
from functools import reduce
import xgboost as xgb
# from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
# from skmultilearn.model_selection import MultiLabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [9]:
# breed
feat_breed = pd.read_pickle('../features/breed.pkl')
feat_breed_v2 = pd.read_pickle('../features/breed_v2.pkl')

feat_breed_top50 = pd.read_pickle('../features/breed_top50.pkl')
feat_breed_top50_v2 = pd.read_pickle('../features/breed_top50_v2.pkl')

feat_breed_group = pd.read_pickle('../features/breed_group.pkl')
feat_sub_breed = pd.read_pickle('../features/sub_breed.pkl')
feat_breed_type = pd.read_pickle('../features/breed_type.pkl')

feat_breed_pure_or_mix = pd.read_pickle('../features/breed_pure_or_mix.pkl')

# age
feat_age = pd.read_csv('../features/age_with_id.csv')

# sex
feat_sex = pd.read_csv('../features/one_hot_encoded_sex_with_id.csv')

# weight
feat_weight = pd.read_pickle('../features/weight.pkl')

# Climate
# feat_HotWheater = pd.read_csv('../features/one_hot_encoded_HotWheater_with_id.csv')
# feat_ModerateWheather = pd.read_csv('../features/one_hot_encoded_ModerateWheather_with_id.csv')
# feat_ColdWheater_with_id = pd.read_csv('../features/one_hot_encoded_ColdWheater_with_id.csv')
feat_cold_month = pd.read_pickle('../features/ColdMonths.pkl')
feat_hot_month = pd.read_pickle('../features/HotMonths.pkl')

# Diet
feat_diet = pd.read_pickle('../features/diet.pkl') # 5 columns

# Physical Activity
feat_pa_total_hours = pd.read_csv('../features/PhysicalActivity_total_hours.csv')
feat_pa_surface = pd.read_csv('../features/PhysicalActivity_surface.csv')
feat_pa_wheather = pd.read_csv('../features/PhysicalActivity_wheather.csv')

# Owner Demographics
feat_od_income = pd.read_pickle('../features/od_income.pkl')
feat_od_education = pd.read_pickle('../features/od_education.pkl')

# Residentual
feat_prim_census_division = pd.read_pickle('../features/primary_residence_census_division.pkl')

# disease
#feat_disease_input = pd.read_csv('../features/one_hot_encoded_disease_input.csv')
feat_disease_output_binary = pd.read_csv('../features/disease_output_binary.csv')
feat_disease_output = pd.read_csv('../features/disease_output.csv')

# age_condition
feat_age_condition = pd.read_pickle('../features/age_condition.pkl')
feat_age_condition_type = pd.read_pickle('../features/age_condition_type.pkl')


features_list = [

    # breed
    #feat_breed,
    #feat_breed_v2,
    #feat_breed_top50,
    #feat_breed_top50_v2,
    #feat_breed_group,
    #feat_sub_breed,
    #feat_breed_type,
    #feat_breed_pure_or_mix,

    # age
    #feat_age,   # 24881 dog_ids

    # sex
    #feat_sex,

    # weight
    #feat_weight

    # diet
    #feat_diet,  # 33141 dog_ids for df_diet_consistency

    #feat_HotWheater,
    #feat_ModerateWheather,
    #feat_ColdWheater_with_id,

    # Climate
    #feat_pa_surface,
    #feat_pa_total_hours,
    #feat_pa_wheather,   # 26406 dog_ids
    #feat_hot_month,
    #feat_cold_month,

    # Physical Activity
    #feat_pa_total_hours,
    #feat_pa_surface,

    # Owner demographics
    #feat_od_income,    # 29096 dog_ids
    #feat_od_education

    # Residentual
    feat_prim_census_division,

    # disease
    #feat_disease_input,
    #feat_disease_output_binary,
    #feat_disease_output
]

feat_age_condition_type

,dog_id,age_diagnosis_years,condition_type_Bone/Orthopedic,condition_type_Brain/Neurologic,condition_type_Cardiac,condition_type_Ear/Nose/Throat,condition_type_Endocrine,condition_type_Eye,condition_type_Gastrointestinal,condition_type_Hematopoietic,...,condition_type_Kidney/Urinary,condition_type_Liver/Pancreas,condition_type_Mouth/Dental/Oral,condition_type_Other Congenital Disorder,condition_type_Reproductive,condition_type_Respiratory,condition_type_Skin,condition_type_Toxin Consumption,condition_type_Trauma,condition_type_cancer
0,100001,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,100001,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,100001,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,100001,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,100001,4,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910915,3373,10,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910929,14856,10,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910936,58327,6,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910945,3417,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [4]:
exp1 = [feat_breed_group]
exp2 = [feat_breed_top50_v2]
exp3 = [feat_age]
exp4 = [feat_sex,feat_breed_type]
exp5 = [feat_sub_breed]
exp6 = [feat_breed_pure_or_mix]
exp7 = [feat_breed_top50_v2,feat_breed_group,feat_weight]
exp8 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex]
exp9 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather]
exp10 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division]
exp11 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_od_education]
exp12 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather]
exp13 = [feat_breed_top50_v2,feat_breed_group,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather]
exp14 = [feat_breed_top50_v2,feat_breed_group,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_diet]
exp15 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_diet]
exp16 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]
exp17 = [feat_breed_top50_v2,feat_breed_group,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet,feat_weight]
exp18 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]
exp19 = [feat_breed_top50_v2,feat_breed_group,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_sub_breed,feat_diet,feat_weight]
exp20 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet,feat_weight,feat_age,feat_sex]

In [5]:
experiments = {
    'exp1': exp1,
    'exp2': exp2,
    'exp3': exp3,
    'exp4': exp4,
    'exp5': exp5,
    'exp6': exp6,
    'exp7': exp7,
    'exp8': exp8,
    'exp9': exp9,
    'exp10': exp10,
    'exp11': exp11,
    'exp12': exp12,
    'exp13': exp13,
    'exp14': exp14,
    'exp15': exp15,
    'exp16': exp16,
    'exp17': exp17,
    'exp18': exp18,
    'exp19': exp19,
    'exp20': exp20,
}


 Richness of Feauters on Normal Dataset

In [6]:
results_auc_train = {}
results_auc_val = {}
results_accuracy = {}
for exp_name, exp_features in experiments.items():
    # List of DataFrames to be merged
    list_input_features = features_list  # Add more DataFrames as needed

    # Merge DataFrames iteratively using reduce
    input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), exp_features)

    # Merge with disease output feature
    data = pd.merge(input_features, feat_disease_output_binary, on='dog_id')
    # data = pd.merge(feat_age_condition_type, input_features, on='dog_id')
    # Assuming 'data' is your DataFrame
    rows_with_nan = data[data.isna().any(axis=1)]
    columns_with_nan = data.columns[data.isna().any()].tolist()
    # Display the rows with NaN values
    rows_with_nan[columns_with_nan]

    # Normalize the specified features in one line and save in the same columns
    features_to_normalize = ['age_diagnosis_years', 'df_diet_consistency', 'df_feedings_per_day', 'total_active_hours_y', 'pa_hot_weather_months_per_year', 'pa_cold_weather_months_per_year', 'total_active_hours_y', 'od_annual_income_range_usd']



    # select features that are in data
    features_to_normalize_in_data = [feature for feature in features_to_normalize if feature in data.columns]

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Normalize
    if features_to_normalize_in_data:
        data[features_to_normalize_in_data] = scaler.fit_transform(data[features_to_normalize_in_data])
        # print(data[features_to_normalize_in_data])


    # Separate features and labels
    X = data.drop(['dog_id'] + ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]], axis=1)

    y_columns = ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]]
#     X = data.drop(['dog_id'] + ['condition_type_' + condition_type for condition_type in [
#     'Eye', 'Ear/Nose/Throat', 'Mouth/Dental/Oral', 'Skin', 'Cardiac', 'Respiratory',
#     'Gastrointestinal', 'Liver/Pancreas', 'Kidney/Urinary', 'Reproductive', 'Bone/Orthopedic',
#     'Brain/Neurologic', 'Endocrine', 'Hematopoietic', 'Other Congenital Disorder',
#     'Infection/Parasites', 'Toxin Consumption', 'Trauma', 'Immune-mediated', 'cancer'
# ]], axis=1)

#     y_columns = ['condition_type_' + condition_type for condition_type in [
#     'Eye', 'Ear/Nose/Throat', 'Mouth/Dental/Oral', 'Skin', 'Cardiac', 'Respiratory',
#     'Gastrointestinal', 'Liver/Pancreas', 'Kidney/Urinary', 'Reproductive', 'Bone/Orthopedic',
#     'Brain/Neurologic', 'Endocrine', 'Hematopoietic', 'Other Congenital Disorder',
#     'Infection/Parasites', 'Toxin Consumption', 'Trauma', 'Immune-mediated', 'cancer'
# ]]
    y = data[y_columns]

    # Convert y to a binary format
    y_binary = (y == 1)

    # Initialize the Naive Bayes model
    model = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, reg_lambda=1, reg_alpha=0.1, subsample=0.8, colsample_bytree=0.8)

    # Wrap the model with OneVsRestClassifier
    ovr_classifier = OneVsRestClassifier(model)

    # Initialize MultilabelStratifiedKFold
    n_splits = 5  # You can adjust the number of splits as needed
    ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    accuracy_scores_per_fold = []
    auc_scores_per_fold_train = []
    auc_scores_per_fold_val = []
    # Lists to store AUC scores and sample counts
    auc_scores_per_condition = {condition: [] for condition in y_columns}
    sample_counts_per_condition = {condition: [] for condition in y_columns}

    # Iterate through the splits
    for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X, y_binary)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

        # Train the model
        ovr_classifier.fit(X_train, y_train)

        # Make predictions on the validation set
        y_pred_proba = ovr_classifier.predict_proba(X_val)
        y_pred_binary = (y_pred_proba >= 0.5).astype(int)

        # Make predictions on the training and validation set
        y_pred_proba_train = ovr_classifier.predict_proba(X_train)
        y_pred_proba_val = ovr_classifier.predict_proba(X_val)

        # Calculate the AUC score for each disease on the training set
        auc_scores_train = [roc_auc_score(y_train[column], y_pred_proba_train[:, i]) for i, column in enumerate(y_train.columns)]
        auc_scores_per_fold_train.append(np.mean(auc_scores_train))

        # Calculate the AUC score for each disease on the validation set
        auc_scores_val = [roc_auc_score(y_val[column], y_pred_proba_val[:, i]) for i, column in enumerate(y_val.columns)]
        auc_scores_per_fold_val.append(np.mean(auc_scores_val))

        # Calculate the AUC score for each disease
        auc_scores = [roc_auc_score(y_val[column], y_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]

        # # get sample counts
        # for i, auc_score in enumerate(auc_scores, start=1):
        #     print(f"{y_val.columns[i-1]}: {auc_score}")
        #     auc_scores_per_condition[y_val.columns[i-1]].append(auc_score)

        if exp_name == 'exp20':
            exp20_auc_scores_per_condition = auc_scores_per_condition.copy()

    average_auc_score_train = np.mean(auc_scores_per_fold_train)
    average_auc_score_val = np.mean(auc_scores_per_fold_val)
    results_auc_train[exp_name] = average_auc_score_train
    results_auc_val[exp_name] = average_auc_score_val

    print("Training Set AUC Scores:")
    print(f"{exp_name}: Average AUC = {average_auc_score_train:.4f}")
    print("\nValidation Set AUC Scores:")
    print(f"{exp_name}: Average AUC = {average_auc_score_val:.4f}")


# Print the results
print("Training Set AUC Scores:")
for exp_name, auc_score in results_auc_train.items():
    print(f"{exp_name}: Average AUC = {auc_score:.4f}")

print("\nValidation Set AUC Scores:")
for exp_name, auc_score in results_auc_val.items():
    print(f"{exp_name}: Average AUC = {auc_score:.4f}")

# for exp_name, accuracy_score in results_accuracy.items():
#     print(f"{exp_name}: Average Accuracy = {accuracy_score:.4f}")


Training Set AUC Scores:
exp1: Average AUC = 0.5792

Validation Set AUC Scores:
exp1: Average AUC = 0.5629
Training Set AUC Scores:
exp2: Average AUC = 0.6147

Validation Set AUC Scores:
exp2: Average AUC = 0.5769
Training Set AUC Scores:
exp3: Average AUC = 0.6789

Validation Set AUC Scores:
exp3: Average AUC = 0.6678
Training Set AUC Scores:
exp4: Average AUC = 0.5796

Validation Set AUC Scores:
exp4: Average AUC = 0.5590
Training Set AUC Scores:
exp5: Average AUC = 0.5873

Validation Set AUC Scores:
exp5: Average AUC = 0.5664
Training Set AUC Scores:
exp6: Average AUC = 0.5241

Validation Set AUC Scores:
exp6: Average AUC = 0.5227
Training Set AUC Scores:
exp7: Average AUC = 0.6584

Validation Set AUC Scores:
exp7: Average AUC = 0.5892
Training Set AUC Scores:
exp8: Average AUC = 0.7375

Validation Set AUC Scores:
exp8: Average AUC = 0.6985
Training Set AUC Scores:
exp9: Average AUC = 0.7608

Validation Set AUC Scores:
exp9: Average AUC = 0.6979
Training Set AUC Scores:
exp10: Avera

Train model on Extend Dataset

In [19]:
results_auc_train = {}
results_auc_val = {}
results_accuracy = {}
exp20 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet,feat_weight,feat_sex]
experiments = {
    'exp20': exp20
    }
for exp_name, exp_features in experiments.items():
    if exp_name != 'exp20':
        continue
    # List of DataFrames to be merged
    list_input_features = features_list  # Add more DataFrames as needed

    # Merge DataFrames iteratively using reduce
    input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), exp_features)

    # Merge with disease output feature
    # data = pd.merge(input_features, feat_disease_output_binary, on='dog_id')
    data = pd.merge(feat_age_condition_type, input_features, on='dog_id')
    # Assuming 'data' is your DataFrame
    rows_with_nan = data[data.isna().any(axis=1)]
    columns_with_nan = data.columns[data.isna().any()].tolist()
    # Display the rows with NaN values
    rows_with_nan[columns_with_nan]

    # Normalize the specified features in one line and save in the same columns
    features_to_normalize = ['age_diagnosis_years', 'df_diet_consistency', 'df_feedings_per_day', 'total_active_hours_y', 'pa_hot_weather_months_per_year', 'pa_cold_weather_months_per_year', 'total_active_hours_y', 'od_annual_income_range_usd']

    # select features that are in data
    features_to_normalize_in_data = [feature for feature in features_to_normalize if feature in data.columns]

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Normalize
    if features_to_normalize_in_data:
        data[features_to_normalize_in_data] = scaler.fit_transform(data[features_to_normalize_in_data])
        # print(data[features_to_normalize_in_data])
    # Separate features and labels
    # X = data.drop(['dog_id'] + ['hs_health_conditions_' + condition for condition in [
    #     'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
    #     'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
    #     'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    # ]], axis=1)

    # y_columns = ['hs_health_conditions_' + condition for condition in [
    #     'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
    #     'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
    #     'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    # ]]
    X = data.drop(['dog_id'] + ['condition_type_' + condition_type for condition_type in [
    'Eye', 'Ear/Nose/Throat', 'Mouth/Dental/Oral', 'Skin', 'Cardiac', 'Respiratory',
    'Gastrointestinal', 'Liver/Pancreas', 'Kidney/Urinary', 'Reproductive', 'Bone/Orthopedic',
    'Brain/Neurologic', 'Endocrine', 'Hematopoietic', 'Other Congenital Disorder',
    'Infection/Parasites', 'Toxin Consumption', 'Trauma', 'Immune-mediated', 'cancer'
]], axis=1)

    y_columns = ['condition_type_' + condition_type for condition_type in [
    'Eye', 'Ear/Nose/Throat', 'Mouth/Dental/Oral', 'Skin', 'Cardiac', 'Respiratory',
    'Gastrointestinal', 'Liver/Pancreas', 'Kidney/Urinary', 'Reproductive', 'Bone/Orthopedic',
    'Brain/Neurologic', 'Endocrine', 'Hematopoietic', 'Other Congenital Disorder',
    'Infection/Parasites', 'Toxin Consumption', 'Trauma', 'Immune-mediated', 'cancer'
]]
    y = data[y_columns]

    # Convert y to a binary format
    y_binary = (y == 1)

    # Initialize the Naive Bayes model
    model = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, reg_lambda=1, reg_alpha=0.1, subsample=0.8, colsample_bytree=0.8)

    # Wrap the model with OneVsRestClassifier
    ovr_classifier = OneVsRestClassifier(model)

    # Initialize MultilabelStratifiedKFold
    n_splits = 5  # You can adjust the number of splits as needed
    ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    accuracy_scores_per_fold = []
    auc_scores_per_fold_train = []
    auc_scores_per_fold_val = []
    # Lists to store AUC scores and sample counts
    auc_scores_per_condition = {condition: [] for condition in y_columns}
    sample_counts_per_condition = {condition: [] for condition in y_columns}

    # Iterate through the splits
    for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X, y_binary)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

        # Train the model
        ovr_classifier.fit(X_train, y_train)

        # Make predictions on the validation set
        y_pred_proba = ovr_classifier.predict_proba(X_val)
        y_pred_binary = (y_pred_proba >= 0.5).astype(int)

        # Make predictions on the training and validation set
        y_pred_proba_train = ovr_classifier.predict_proba(X_train)
        y_pred_proba_val = ovr_classifier.predict_proba(X_val)

        # Calculate the AUC score for each disease on the training set
        auc_scores_train = [roc_auc_score(y_train[column], y_pred_proba_train[:, i]) for i, column in enumerate(y_train.columns)]
        auc_scores_per_fold_train.append(np.mean(auc_scores_train))

        # Calculate the AUC score for each disease on the validation set
        auc_scores_val = [roc_auc_score(y_val[column], y_pred_proba_val[:, i]) for i, column in enumerate(y_val.columns)]
        auc_scores_per_fold_val.append(np.mean(auc_scores_val))

        # Calculate the AUC score for each disease
        auc_scores = [roc_auc_score(y_val[column], y_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]

        if exp_name == 'exp20':
            
            for i, column in enumerate(y_val.columns):
                auc_score = roc_auc_score(y_val[column], y_pred_proba[:, i])
                auc_scores_per_condition[column].append(auc_score)

            
            exp20_auc_scores_per_condition = auc_scores_per_condition.copy()
    
    average_auc_score_train = np.mean(auc_scores_per_fold_train)
    average_auc_score_val = np.mean(auc_scores_per_fold_val)
    results_auc_train[exp_name] = average_auc_score_train
    results_auc_val[exp_name] = average_auc_score_val



# Print the results
print("Training Set AUC Scores:")
for exp_name, auc_score in results_auc_train.items():
    print(f"{exp_name}: Average AUC = {auc_score:.4f}")

print("\nValidation Set AUC Scores:")
for exp_name, auc_score in results_auc_val.items():
    print(f"{exp_name}: Average AUC = {auc_score:.4f}")


Training Set AUC Scores:
exp20: Average AUC = 0.7764

Validation Set AUC Scores:
exp20: Average AUC = 0.7040


In [20]:
if exp20_auc_scores_per_condition:
    print(f"\nAUC Scores for Each Condition in exp20:")
    for condition, auc_scores in exp20_auc_scores_per_condition.items():
        average_auc = sum(auc_scores) / len(auc_scores) if auc_scores else 0
        print(f"{condition}: {average_auc * 100:.2f}%")


AUC Scores for Each Condition in exp20:
condition_type_Eye: 71.08%
condition_type_Ear/Nose/Throat: 71.36%
condition_type_Mouth/Dental/Oral: 71.05%
condition_type_Skin: 64.22%
condition_type_Cardiac: 73.16%
condition_type_Respiratory: 74.47%
condition_type_Gastrointestinal: 66.42%
condition_type_Liver/Pancreas: 69.48%
condition_type_Kidney/Urinary: 71.48%
condition_type_Reproductive: 78.25%
condition_type_Bone/Orthopedic: 66.71%
condition_type_Brain/Neurologic: 67.99%
condition_type_Endocrine: 69.74%
condition_type_Hematopoietic: 67.42%
condition_type_Other Congenital Disorder: 82.55%
condition_type_Infection/Parasites: 76.55%
condition_type_Toxin Consumption: 63.93%
condition_type_Trauma: 64.26%
condition_type_Immune-mediated: 62.95%
condition_type_cancer: 75.00%


Save Model as .joblib

In [21]:
model_filepath = '../models/extreme_gradient_boosting.joblib'
joblib.dump(ovr_classifier, model_filepath)

['../models/extreme_gradient_boosting.joblib']

Test Model

In [22]:
loaded_model_filepath = '../models/extreme_gradient_boosting.joblib'
loaded_ovr_classifier = joblib.load(loaded_model_filepath)

In [23]:
# Assuming X_val is your validation set DataFrame
# Select a random row (Datapoint) from X_val
random_index = np.random.choice(X_val.index)

Show input

In [26]:
import pandas as pd

pd.set_option('display.max_rows', None)

random_data_point = X_val.loc[random_index]
random_data_point

age_diagnosis_years                            0.36
breeds_American Pitbull Terrier                   0
breeds_American Staffordshire Terrier             0
breeds_Australian Cattle Dog                      0
breeds_Australian Shepherd                        0
breeds_Basset Hound                               0
breeds_Beagle                                     0
breeds_Bernese Mountain Dog                       0
breeds_Bichon Frise                               0
breeds_Border Collie                              0
breeds_Boston Terrier                             0
breeds_Boxer                                      0
breeds_Bulldog                                    0
breeds_Cairn Terrier                              0
breeds_Catahoula Leopard Dog                      0
breeds_Cavalier King Charles Spaniel              0
breeds_Chihuahua                                  0
breeds_Chow Chow                                  0
breeds_Cocker Spaniel                             0
breeds_Colli

In [25]:
# Reshape the data point to a 2D array
random_data_point_reshaped = random_data_point.values.reshape(1, -1)

# Make predictions on the reshaped data point using the loaded model
new_data_predictions_proba = loaded_ovr_classifier.predict_proba(random_data_point_reshaped)
new_data_predictions_proba

array([[2.9448701e-02, 9.5726280e-03, 1.3874164e-01, 4.5569826e-02,
        1.7140659e-02, 3.9018071e-03, 1.4330287e-02, 4.8306636e-03,
        1.5845800e-02, 9.3036302e-04, 3.1049768e-02, 3.7427917e-03,
        4.4639255e-03, 2.1266640e-04, 9.5680509e-05, 9.6346075e-03,
        1.0239634e-02, 1.9094439e-02, 1.5415238e-03, 6.4909840e-03]],
      dtype=float32)